In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,25229
2,Huelva,Confirmados PDIA 14 días,471
3,Huelva,Tasa PDIA 14 días,"91,78245026014771"
4,Huelva,Confirmados PDIA 7 días,244
5,Huelva,Total Confirmados,25429
6,Huelva,Curados,20276
7,Huelva,Fallecidos,349


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  25229.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6656.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 5137 personas en los últimos 7 días 

Un positivo PCR cada 2115 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,25229.0,244.0,471.0,513170.0,47.547596,91.782450,20.0
Condado-Campiña,7204.0,143.0,258.0,156231.0,91.531130,165.140081,16.0
Rociana del Condado,341.0,20.0,29.0,7939.0,251.920897,365.285300,8.0
Bollullos Par del Condado,646.0,15.0,27.0,14387.0,104.260791,187.669424,4.0
Niebla,165.0,7.0,8.0,4116.0,170.068027,194.363460,4.0
Huelva (capital),6656.0,28.0,68.0,143837.0,19.466479,47.275736,2.0
Huelva-Costa,15295.0,89.0,189.0,289548.0,30.737563,65.274151,2.0
Gibraleón,535.0,13.0,22.0,12737.0,102.064850,172.725132,1.0
Sierra de Huelva-Andévalo Central,2312.0,4.0,13.0,67391.0,5.935511,19.290410,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Alosno,145.0,20.0,23.0,3933.0,508.517671,584.795322,0.0
Villalba del Alcor,459.0,4.0,13.0,3366.0,118.835413,386.215092,0.0
Rociana del Condado,341.0,20.0,29.0,7939.0,251.920897,365.285300,8.0
Puebla de Guzmán,110.0,2.0,11.0,3092.0,64.683053,355.756792,0.0
San Bartolomé de la Torre,216.0,6.0,13.0,3761.0,159.532039,345.652752,0.0
Almonte,955.0,53.0,80.0,24507.0,216.264741,326.437344,0.0
Niebla,165.0,7.0,8.0,4116.0,170.068027,194.363460,4.0
Bollullos Par del Condado,646.0,15.0,27.0,14387.0,104.260791,187.669424,4.0
Gibraleón,535.0,13.0,22.0,12737.0,102.064850,172.725132,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Puebla de Guzmán,110.0,2.0,11.0,3092.0,64.683053,355.756792,0.0,0.181818
Ayamonte,1237.0,5.0,17.0,21104.0,23.692191,80.553450,0.0,0.294118
Valverde del Camino,426.0,3.0,10.0,12750.0,23.529412,78.431373,1.0,0.300000
Villalba del Alcor,459.0,4.0,13.0,3366.0,118.835413,386.215092,0.0,0.307692
Sierra de Huelva-Andévalo Central,2312.0,4.0,13.0,67391.0,5.935511,19.290410,1.0,0.307692
Palos de la Frontera,465.0,4.0,13.0,11742.0,34.065747,110.713677,0.0,0.307692
Lepe,2011.0,6.0,16.0,27880.0,21.520803,57.388809,0.0,0.375000
Huelva (capital),6656.0,28.0,68.0,143837.0,19.466479,47.275736,2.0,0.411765
San Bartolomé de la Torre,216.0,6.0,13.0,3761.0,159.532039,345.652752,0.0,0.461538
